This notebook was executed in Google Colab using CPU

### Start of execution

In [1]:
import time

In [2]:
start = time.time()

# 1. Setting the environment

In [3]:
!pip install -q datasets==2.20.0

# 2. Import libraries

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import create_optimizer
from transformers import DataCollatorWithPadding
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# 3. Preparation

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path_general = 'drive/MyDrive/Profesional_Academico/Github_Personal/ML_AI_Contents/09.Deep_Learning/51.BERT_HF_TensorFlow'

# 3. Load Dataset

In [8]:
imdb = load_dataset("imdb")

In [9]:
df_train = imdb['train'].to_pandas()

In [10]:
df_test = imdb['test'].to_pandas()

# 4. Load model

In [11]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
model.load_weights(f'{path_general}/model/model')

# 5. Inference

In [14]:
def predict_text(text, tokenizer):

  encoded_input = tokenizer(text, return_tensors = 'tf', truncation = True)
  output = model(**encoded_input)
  logits_array = output.logits.numpy()[0]

  if logits_array[0] > logits_array[1]:
    return 0

  else:
    return 1

In [15]:
df_train = df_train.sample(1_000)

In [16]:
df_train['pred'] = df_train['text'].map(lambda x: predict_text(x, tokenizer))

In [17]:
df_test = df_test.sample(1_000)

In [18]:
df_test['pred'] = df_test['text'].map(lambda x: predict_text(x, tokenizer))

# 6. Performance

In [19]:
from sklearn.metrics import recall_score, precision_score, f1_score

### a. Train

In [20]:
round(precision_score(df_train['label'], df_train['pred'], average = 'macro')*100, 2)

98.71

In [21]:
round(recall_score(df_train['label'], df_train['pred'], average = 'macro')*100, 2)

98.69

In [22]:
round(f1_score(df_train['label'], df_train['pred'], average = 'macro')*100, 2)

98.7

In [23]:
round(((df_train['label'] == df_train['pred'])*1).mean()*100, 2)

98.7

### b. Test

In [24]:
round(precision_score(df_test['label'], df_test['pred'], average = 'macro')*100, 2)

94.4

In [25]:
round(recall_score(df_test['label'], df_test['pred'], average = 'macro')*100, 2)

94.39

In [26]:
round(f1_score(df_test['label'], df_test['pred'], average = 'macro')*100, 2)

94.39

In [27]:
round(((df_test['label'] == df_test['pred'])*1).mean()*100, 2)

94.4

### End of execution

In [28]:
end = time.time()

delta = (end - start)

hours = int(delta/3_600)
mins = int((delta - hours*3_600)/60)
secs = int(delta - hours*3_600 - mins*60)

print(f'Hours: {hours}, Minutes: {mins}, Seconds: {secs}')

Hours: 0, Minutes: 6, Seconds: 37
